In [1]:
# Import necessary packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import Binarizer, LabelEncoder, MultiLabelBinarizer, OneHotEncoder
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score, precision_recall_curve, accuracy_score, precision_score, f1_score, ConfusionMatrixDisplay, confusion_matrix
from scipy.stats import ttest_rel, ttest_ind
from scipy import stats
from keras.layers import Input, Conv1D, MaxPooling1D, AveragePooling1D, Flatten, Dense, Dropout
from keras.models import Model, load_model
from keras import backend as K
import tensorflow as tf
import cms_preprocess

2023-03-16 12:39:34.565619: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-16 12:39:34.612769: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/local-scratch/localhome/ada104/orx/cms_preprocess.py:17: DtypeWarning: Columns (1,3,21) have mixed types. Specify dtype option on import or set low_memory=False.
  ben = pd.read_csv("/localhome/ada104/orx/data/cms/ben.csv",
/local-scratch/localhome/ada104/orx/cms_preprocess.py:19: DtypeWarning: Columns (1,3,30) have mixed t

In [3]:
# Split the data into train and test sets
x_train, x_test, y_train, y_test = cms_preprocess.get_aov(beneficiaries_df=cms_preprocess.ben,
                                                 inpatient_df=cms_preprocess.ip, dx = cms_preprocess.dx, pcs= cms_preprocess.pcs, 
                                                              start_year=2008, end_year=2010, random_state = 42)

# Initialize the Lasso classifier
clf = Lasso(alpha=0.001)

# 5-fold cross validation
scores = cross_val_score(clf, x_train, y_train, cv=5)

# Mean and standard deviation of the cross validation scores
print("Mean:", np.mean(scores))
print("Standard deviation:", np.std(scores))

# Fit the classifier to the training data
clf.fit(x_train, y_train)

# Predict the target values on the test set
y_pred = clf.predict(x_test)

# Convert the continuous predictions to binary classes
y_pred_binary = [1 if y > 0.5 else 0 for y in y_pred]

# Evaluate the classifier performance on the test set
print("Accuracy:", accuracy_score(y_test, y_pred_binary))
print("AUC:", roc_auc_score(y_test, y_pred_binary))

# calculate the precision recall curve
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_binary)

# find the closest recall value to 0.9
closest_idx = (np.abs(recall-0.9)).argmin()
precision_at_recall_0_9 = precision[closest_idx]

print("Precision with recall set to 0.9:", precision_at_recall_0_9)

ValueError: cannot reshape array of size 21435408 into shape (1572)